In [1]:
import pandas as pd
import numpy as np
import json
import os
import datetime
from haversine import haversine
import zipcodes
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

# S 20 miles

In [2]:
store_address_SFTP=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190401-134939-117.txt",dtype=str,sep="|")
store_address_SFTP=store_address_SFTP[['location_id','latitude_meas','longitude_meas','zip_cd']]

store_address_SFTP['latitude_meas']=store_address_SFTP['latitude_meas'].astype(float)
store_address_SFTP['longitude_meas']=store_address_SFTP['longitude_meas'].astype(float)
store_address_SFTP['zip_cd']=store_address_SFTP['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))

store_address_SFTP=store_address_SFTP.rename(columns={"zip_cd":"store_zip"})

In [3]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425'

In [4]:
store_quadrants=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425/Excel_BL_2018_Q4_post_YoY_small_JL_2019-03-04.xlsx",
                             dtype=str,sheetname="Q4_Store_Quadrant_Defination",usecols=["location_id",'Quadrant'])
print(store_quadrants.shape)


(1361, 2)


In [5]:
SOTF_list=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425/BL Store List Report 02.28.19 with DMA and SOTF 20190415.xlsx",
                       dtype=str,sheetname="SOTF 20190415")
print(SOTF_list.shape)
print(len(SOTF_list['Store'].unique()))
SOTF_list=SOTF_list.rename(columns={"Store":"location_id"})

(203, 22)
203


In [6]:
store_zip_label=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_zip_label=store_zip_label[['location_id','zip','revenue_flag']].drop_duplicates()

store_zip_label=store_zip_label.rename(columns={"zip":"zip_cd"})
store_zip_label=store_zip_label[store_zip_label['revenue_flag']!="T"]
store_zip_label['revenue_flag'].unique()

array(['P', 'S'], dtype=object)

In [7]:
Open_Stores=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-04-20.csv",dtype=str,usecols=['location_id','week_end_date','sales'])
Open_Stores['sales']=Open_Stores['sales'].astype(float)
Open_Stores=Open_Stores[Open_Stores['sales']>0]
Open_Stores=Open_Stores[Open_Stores['week_end_date'].isin(['2019-04-20','2019-04-13','2019-04-06','2019-03-30'])]
Open_Stores=Open_Stores.groupby(['location_id'])['sales'].count().to_frame().reset_index().rename(columns={"sales":"recent_month_open_weeks"})
print(Open_Stores.shape)

Open_Stores=Open_Stores[Open_Stores['recent_month_open_weeks']>=3]
print(Open_Stores.shape)
Open_Stores['recent_month_open_weeks']="Open"
Open_Stores=Open_Stores.rename(columns={"recent_month_open_weeks":"open_label"})
Open_Stores.head(2)
# Only the quadrant zip-stores will be applied to the opening store filter, and apply the fillna for quadrant III

(1402, 2)
(1399, 2)


,location_id,open_label
0,1,Open
1,1001,Open


In [8]:
store_quadrants.head(2)

,location_id,Quadrant
0,1,Quadrant I
1,3,Quadrant I


In [9]:
store_zip_label.head(2)

,location_id,zip_cd,revenue_flag
0,1,43232,P
1,1,43209,P


In [10]:
Open_Stores.head(2)

,location_id,open_label
0,1,Open
1,1001,Open


# Quadrant All

In [11]:
store_quadrants=pd.merge(Open_Stores,store_quadrants,on="location_id",how="left")
store_quadrants['Quad_defined_method']=np.where(pd.isnull(store_quadrants['Quadrant']),"Filled_with_Quad_III","2018Q4_defined")
store_quadrants['Quadrant']=store_quadrants['Quadrant'].fillna("Quadrant III")
store_quadrants_zips=pd.merge(store_quadrants,store_zip_label,on="location_id",how="left")
store_quadrants_zips.head(4)

print(len(store_quadrants_zips['location_id'].unique()))

store_quadrants_zips_new_stores=store_quadrants_zips[pd.isnull(store_quadrants_zips['revenue_flag'])]
print(store_quadrants_zips_new_stores.shape)

store_quadrants_zips_old_stores=store_quadrants_zips[~pd.isnull(store_quadrants_zips['revenue_flag'])]

1399
(62, 6)


In [12]:
store_quadrants=pd.merge(store_quadrants,store_address_SFTP,on="location_id",how="left")
print(store_quadrants.shape)
print(store_quadrants[pd.isnull(store_quadrants['latitude_meas'])]['location_id'].unique())

dict_getting_address={}
dict_getting_address.update({"4674":{"GoogleAddress":"8501 Auburn Blvd, Citrus Heights, CA 95610","GoogleCoor":[38.7211825,-121.2927472]}})
dict_getting_address.update({"4693":{"GoogleAddress":"11696 NE 76th St, Vancouver, WA 98662","GoogleCoor":[45.6788111,-122.5536852]}})

store_quadrants.loc[store_quadrants['location_id']=="4674",'latitude_meas']=dict_getting_address['4674']['GoogleCoor'][0]
store_quadrants.loc[store_quadrants['location_id']=="4674",'longitude_meas']=dict_getting_address['4674']['GoogleCoor'][1]
store_quadrants.loc[store_quadrants['location_id']=="4674",'store_zip']='95610'

store_quadrants.loc[store_quadrants['location_id']=="4693",'latitude_meas']=dict_getting_address['4693']['GoogleCoor'][0]
store_quadrants.loc[store_quadrants['location_id']=="4693",'longitude_meas']=dict_getting_address['4693']['GoogleCoor'][1]
store_quadrants.loc[store_quadrants['location_id']=="4693",'store_zip']='98662'

store_quadrants=store_quadrants[~pd.isnull(store_quadrants['latitude_meas'])]
print(store_quadrants.shape)


(1399, 7)
['4674' '4693']
(1399, 7)


In [13]:
store_quadrants_zips_new_stores.shape

(62, 6)

In [14]:
store_quadrants_zips_new_stores.head(2)

,location_id,open_label,Quadrant,Quad_defined_method,zip_cd,revenue_flag
17517,4667,Open,Quadrant III,2018Q4_defined,NaN,NaN
17518,4669,Open,Quadrant III,2018Q4_defined,NaN,NaN


In [15]:
del store_quadrants_zips_new_stores['zip_cd']
del store_quadrants_zips_new_stores['revenue_flag']

store_quadrants_zips_new_stores=pd.merge(store_quadrants_zips_new_stores,store_address_SFTP,on="location_id",how="left")

store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4674",'latitude_meas']=dict_getting_address['4674']['GoogleCoor'][0]
store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4674",'longitude_meas']=dict_getting_address['4674']['GoogleCoor'][1]
store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4674",'store_zip']='95610'

store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4693",'latitude_meas']=dict_getting_address['4693']['GoogleCoor'][0]
store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4693",'longitude_meas']=dict_getting_address['4693']['GoogleCoor'][1]
store_quadrants_zips_new_stores.loc[store_quadrants_zips_new_stores['location_id']=="4693",'store_zip']='98662'


print(store_quadrants_zips_new_stores[pd.isnull(store_quadrants_zips_new_stores['latitude_meas'])].shape)
store_quadrants_zips_new_stores=store_quadrants_zips_new_stores.reset_index()
del store_quadrants_zips_new_stores['index']

(0, 7)


In [16]:
df_zips_for_new_quadrant_stores=pd.DataFrame()

for i in range(len(store_quadrants_zips_new_stores)):
    store_num=store_quadrants_zips_new_stores['location_id'][i]
    store_center=[store_quadrants_zips_new_stores['latitude_meas'][i],store_quadrants_zips_new_stores['longitude_meas'][i]]
    
    store_zip_df=pd.DataFrame({"location_id":store_num,'zip_cd':store_quadrants_zips_new_stores['store_zip'][i],"Dist_miles":"store_zip"},index=[0])
    for zip_cd in zip_centers:
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":store_num,'zip_cd':zip_cd,"Dist_miles":dist},index=[0])
            store_zip_df=store_zip_df.append(df)
    df_zips_for_new_quadrant_stores=df_zips_for_new_quadrant_stores.append(store_zip_df)
df_zips_for_new_quadrant_stores['revenue_flag']="zips_in_10"
store_quadrants_zips_new_stores=pd.merge(store_quadrants_zips_new_stores,df_zips_for_new_quadrant_stores,on="location_id",how="left")

In [17]:
store_quadrants_zips_new_stores.head(2)

,location_id,open_label,Quadrant,Quad_defined_method,latitude_meas,longitude_meas,store_zip,Dist_miles,zip_cd,revenue_flag
0,4667,Open,Quadrant III,2018Q4_defined,33.417099,-111.769785,85213,store_zip,85213,zips_in_10
1,4667,Open,Quadrant III,2018Q4_defined,33.417099,-111.769785,85213,8.70104,85244,zips_in_10


In [18]:
store_quadrants_zips_old_stores=pd.merge(store_quadrants_zips_old_stores,store_address_SFTP,on="location_id",how="left")
store_quadrants_zips_old_stores['Dist_miles']=np.nan
store_quadrants_zips_old_stores=store_quadrants_zips_old_stores.reset_index()
del store_quadrants_zips_old_stores['index']

for i in range(len(store_quadrants_zips_old_stores)):
    store_center=[store_quadrants_zips_old_stores['latitude_meas'][i],store_quadrants_zips_old_stores['longitude_meas'][i]]
    zip_cd=store_quadrants_zips_old_stores['zip_cd'][i]
    if zip_cd in zip_centers.keys():
        store_quadrants_zips_old_stores['Dist_miles'][i]=haversine(store_center,zip_centers[zip_cd],miles=True)
    else:
        try:
            zip_center_of_not_in_ren=zipcodes.matching(zip_cd)[0]['lat'],zipcodes.matching(zip_cd)[0]['long']
            store_quadrants_zips_old_stores['Dist_miles'][i]=haversine(store_center,zip_center_of_not_in_ren,miles=True)
        except:
            store_quadrants_zips_old_stores['Dist_miles'][i]=np.nan
            
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [19]:
store_quadrants_zips_old_stores=store_quadrants_zips_old_stores[(store_quadrants_zips_old_stores['revenue_flag']!="S") | (store_quadrants_zips_old_stores['Dist_miles']<=20)]
store_quadrants_zips_old_stores.shape


(20442, 10)

In [20]:
store_quadrants_zips_new_stores.head(2)

,location_id,open_label,Quadrant,Quad_defined_method,latitude_meas,longitude_meas,store_zip,Dist_miles,zip_cd,revenue_flag
0,4667,Open,Quadrant III,2018Q4_defined,33.417099,-111.769785,85213,store_zip,85213,zips_in_10
1,4667,Open,Quadrant III,2018Q4_defined,33.417099,-111.769785,85213,8.70104,85244,zips_in_10


In [21]:
store_quadrants_zips_old_stores.head(2)

,location_id,open_label,Quadrant,Quad_defined_method,zip_cd,revenue_flag,latitude_meas,longitude_meas,store_zip,Dist_miles
0,1,Open,Quadrant I,2018Q4_defined,43232,P,39.913636,-82.914789,43232,2.717654
1,1,Open,Quadrant I,2018Q4_defined,43209,P,39.913636,-82.914789,43232,3.095917


In [22]:
output_quadrants_All=store_quadrants_zips_new_stores.append(store_quadrants_zips_old_stores)
output_quadrants_All=output_quadrants_All[['location_id','Quadrant','store_zip','Quad_defined_method','latitude_meas','longitude_meas','revenue_flag','zip_cd','Dist_miles']]

print(len(output_quadrants_All['location_id'].unique()))
output_quadrants_All.head(2)

1399


,location_id,Quadrant,store_zip,Quad_defined_method,latitude_meas,longitude_meas,revenue_flag,zip_cd,Dist_miles
0,4667,Quadrant III,85213,2018Q4_defined,33.417099,-111.769785,zips_in_10,85213,store_zip
1,4667,Quadrant III,85213,2018Q4_defined,33.417099,-111.769785,zips_in_10,85244,8.70104


# SOTF

In [23]:
SOTF_df=SOTF_list[['location_id']]
SOTF_df=pd.merge(SOTF_df,store_address_SFTP,on="location_id",how="left")

SOTF_df_old=SOTF_df[~pd.isnull(SOTF_df['latitude_meas'])]
print(SOTF_df_old.shape)
SOTF_df_new_no_address=SOTF_df[pd.isnull(SOTF_df['latitude_meas'])]
print(SOTF_df_new_no_address.shape)
SOTF_df_new_no_address

(200, 4)
(3, 4)


,location_id,latitude_meas,longitude_meas,store_zip
134,4693,NaN,NaN,NaN
201,4690,NaN,NaN,NaN
202,5383,NaN,NaN,NaN


In [24]:
df_getting_address=SOTF_list[SOTF_list['location_id'].isin(SOTF_df_new_no_address['location_id'].tolist())]
df_getting_address=df_getting_address[['location_id','Address 1','Address 2','City','ST','Zip']]
df_getting_address['search_key']=df_getting_address['Address 1']+", "+df_getting_address['Address 2']+", "+df_getting_address['City']+", "+df_getting_address['ST']+", "+df_getting_address['Zip']
print(df_getting_address['search_key'].tolist())
# Manually serach google map
dict_getting_address={}
dict_getting_address.update({"4693":{"GoogleAddress":"11696 NE 76th St, Vancouver, WA 98662","GoogleCoor":[45.6788111,-122.5536852]}})
dict_getting_address.update({"4690":{"GoogleAddress":"5636 US-60, Huntington, WV 25705","GoogleCoor":[38.4093679,-82.3245243]}})
dict_getting_address.update({"5383":{"GoogleAddress":"100 Crossroads Plaza, Mt Pleasant, PA 15666","GoogleCoor":[40.1252545,-79.5510085]}})

SOTF_df_new_no_address=SOTF_df_new_no_address.reset_index()

for i in range(len(SOTF_df_new_no_address)):
    store_num=SOTF_df_new_no_address['location_id'][i]
    SOTF_df_new_no_address['latitude_meas'][i]=dict_getting_address[store_num]['GoogleCoor'][0]
    SOTF_df_new_no_address['longitude_meas'][i]=dict_getting_address[store_num]['GoogleCoor'][1]
    SOTF_df_new_no_address['store_zip'][i]=dict_getting_address[store_num]['GoogleAddress'][-5:]
del SOTF_df_new_no_address['index']

['11696 NORTHEAST 76TH STREET, ORCHARD PLAZA, VANCOUVER, WA, 98662-3941', '5636 U.S. 60, PEA RIDGE SHOPPING CENTER, HUNTINGTON, WV, 25705', '100 CROSSROADS PLAZA, KMART SHOPPING CENTER, MOUNT PLEASANT, PA, 15666']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
SOTF_df=SOTF_df_old.append(SOTF_df_new_no_address)

SOTF_df_zips=pd.merge(SOTF_df,store_zip_label,on="location_id",how="left")

SOTF_df_zips_new_stores=SOTF_df_zips[pd.isnull(SOTF_df_zips['revenue_flag'])]
print(SOTF_df_zips_new_stores.shape)

SOTF_df_zips_old_stores=SOTF_df_zips[~pd.isnull(SOTF_df_zips['revenue_flag'])]

SOTF_df_zips_new_stores=SOTF_df_zips_new_stores.reset_index()
del SOTF_df_zips_new_stores['index']

SOTF_df_zips_old_stores=SOTF_df_zips_old_stores.reset_index()
del SOTF_df_zips_old_stores['index']


(46, 6)


In [26]:
df_zips_for_new_SOTF=pd.DataFrame()

del SOTF_df_zips_new_stores['zip_cd']
del SOTF_df_zips_new_stores['revenue_flag']

for i in range(len(SOTF_df_zips_new_stores)):
    store_num=SOTF_df_zips_new_stores['location_id'][i]
    store_center=[SOTF_df_zips_new_stores['latitude_meas'][i],SOTF_df_zips_new_stores['longitude_meas'][i]]
    
    store_zip_df=pd.DataFrame({"location_id":store_num,'zip_cd':SOTF_df_zips_new_stores['store_zip'][i],"Dist_miles":"store_zip"},index=[0])
    for zip_cd in zip_centers:
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":store_num,'zip_cd':zip_cd,"Dist_miles":dist},index=[0])
            store_zip_df=store_zip_df.append(df)
    df_zips_for_new_SOTF=df_zips_for_new_SOTF.append(store_zip_df)
df_zips_for_new_SOTF['revenue_flag']="zips_in_10"
SOTF_df_zips_new_stores=pd.merge(SOTF_df_zips_new_stores,df_zips_for_new_SOTF,on="location_id",how="left")

print(SOTF_df_zips_new_stores.shape)
SOTF_df_zips_new_stores.head(2)

(1165, 7)


,location_id,latitude_meas,longitude_meas,store_zip,Dist_miles,zip_cd,revenue_flag
0,4667,33.417099,-111.769785,85213,store_zip,85213,zips_in_10
1,4667,33.417099,-111.769785,85213,8.70104,85244,zips_in_10


In [27]:
SOTF_df_zips_old_stores['Dist_miles']=np.nan
SOTF_df_zips_old_stores=SOTF_df_zips_old_stores.reset_index()
del SOTF_df_zips_old_stores['index']

for i in range(len(SOTF_df_zips_old_stores)):
    store_center=[SOTF_df_zips_old_stores['latitude_meas'][i],SOTF_df_zips_old_stores['longitude_meas'][i]]
    zip_cd=SOTF_df_zips_old_stores['zip_cd'][i]
    if zip_cd in zip_centers.keys():
        SOTF_df_zips_old_stores['Dist_miles'][i]=haversine(store_center,zip_centers[zip_cd],miles=True)
    else:
        try:
            zip_center_of_not_in_ren=zipcodes.matching(zip_cd)[0]['lat'],zipcodes.matching(zip_cd)[0]['long']
            SOTF_df_zips_old_stores['Dist_miles'][i]=haversine(store_center,zip_center_of_not_in_ren,miles=True)
        except:
            SOTF_df_zips_old_stores['Dist_miles'][i]=np.nan
            
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [28]:
SOTF_df_zips_old_stores=SOTF_df_zips_old_stores[(SOTF_df_zips_old_stores['revenue_flag']!="S") | (SOTF_df_zips_old_stores['Dist_miles']<=20)]
SOTF_df_zips_old_stores.shape


(2750, 7)

In [29]:
output_SOTF=SOTF_df_zips_new_stores.append(SOTF_df_zips_old_stores)
output_SOTF['Quadrant']="SOTF"
output_SOTF['Quad_defined_method']="SOTF"

output_SOTF=output_SOTF[output_quadrants_All.columns.tolist()]



In [35]:
df_output_detailed=output_quadrants_All.append(output_SOTF)

output_Quad1234_zips=df_output_detailed[df_output_detailed['Quadrant']!="SOTF"]
output_Quad1234_zips=output_Quad1234_zips[['zip_cd']].drop_duplicates()
output_Quad1234_zips['zip_type']="unique_of_all_4_quads"

output_Quad_23_zips=df_output_detailed[df_output_detailed['Quadrant'].isin(['Quadrant II','Quadrant III'])]
output_Quad_23_zips=output_Quad_23_zips[['zip_cd']].drop_duplicates()
output_Quad_23_zips['zip_type']="unique_of_all_quad_II_III"

output_Quad_34_zips=df_output_detailed[df_output_detailed['Quadrant'].isin(['Quadrant III','Quadrant IV'])]
output_Quad_34_zips=output_Quad_34_zips[['zip_cd']].drop_duplicates()
output_Quad_34_zips['zip_type']="unique_of_quad_III_IV"


output_Quad1234_SOTF_zips=df_output_detailed[['zip_cd']].drop_duplicates()
output_Quad1234_SOTF_zips['zip_type']="Quads_1-4_and_SOTF"

output_Quad_23_SOTF_zips=df_output_detailed[df_output_detailed['Quadrant'].isin(['Quadrant II','Quadrant III','SOTF'])]
output_Quad_23_SOTF_zips=output_Quad_23_SOTF_zips[['zip_cd']].drop_duplicates()
output_Quad_23_SOTF_zips['zip_type']="Quads_2&3_and_SOTF"

output_Quad_34_SOTF_zips=df_output_detailed[df_output_detailed['Quadrant'].isin(['Quadrant III','Quadrant IV','SOTF'])]
output_Quad_34_SOTF_zips=output_Quad_34_SOTF_zips[['zip_cd']].drop_duplicates()
output_Quad_34_SOTF_zips['zip_type']="Quads_3&4_and_SOTF"

output_SOTF_unique_zip=output_SOTF[['zip_cd','revenue_flag']].drop_duplicates().sort_values(["revenue_flag",'zip_cd']).drop_duplicates("zip_cd")

In [36]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425/BL_zips_for_Quad_1-4_and_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_Quad1234_SOTF_zips.to_excel(writer,"output_Quad1234_SOTF_zips",index=False)
output_SOTF_unique_zip.to_excel(writer,"output_SOTF_only_zips",index=False)
output_Quad1234_zips.to_excel(writer,"output_quad_1-4_only_zips",index=False)
df_output_detailed.to_excel(writer,"All_details",index=False)
writer.save()

In [37]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425/BL_zips_for_Quad_3and4_and_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_Quad_34_SOTF_zips.to_excel(writer,"output_Quad_34_SOTF_zips",index=False)
output_SOTF_unique_zip.to_excel(writer,"output_SOTF_only_zips",index=False)
output_Quad_34_zips.to_excel(writer,"output_quad_34_only_zips",index=False)
df_output_detailed.to_excel(writer,"All_details",index=False)
writer.save()

In [38]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/zips_SOTF_quad_3_gruops_JT_20190425/BL_zips_for_Quad_2and3_and_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_Quad_23_SOTF_zips.to_excel(writer,"output_Quad_23_SOTF_zips",index=False)
output_SOTF_unique_zip.to_excel(writer,"output_SOTF_only_zips",index=False)
output_Quad_23_zips.to_excel(writer,"output_quad_23_only_zips",index=False)
df_output_detailed.to_excel(writer,"All_details",index=False)
writer.save()